In [24]:
import cv2
cap = cv2.VideoCapture(5)
while(1):
    ret, frame = cap.read()
    print(type(ret))
    print(frame.shape)
    cv2.imshow('capture', frame)
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break

<class 'bool'>
(480, 640, 3)
<class 'bool'>
(480, 640, 3)
<class 'bool'>
(480, 640, 3)
<class 'bool'>
(480, 640, 3)
<class 'bool'>
(480, 640, 3)
<class 'bool'>
(480, 640, 3)
<class 'bool'>
(480, 640, 3)
<class 'bool'>
(480, 640, 3)
<class 'bool'>
(480, 640, 3)
<class 'bool'>
(480, 640, 3)
<class 'bool'>
(480, 640, 3)
<class 'bool'>
(480, 640, 3)
<class 'bool'>
(480, 640, 3)
<class 'bool'>
(480, 640, 3)
<class 'bool'>
(480, 640, 3)
<class 'bool'>
(480, 640, 3)
<class 'bool'>
(480, 640, 3)
<class 'bool'>
(480, 640, 3)
<class 'bool'>
(480, 640, 3)
<class 'bool'>
(480, 640, 3)
<class 'bool'>
(480, 640, 3)
<class 'bool'>
(480, 640, 3)
<class 'bool'>
(480, 640, 3)
<class 'bool'>
(480, 640, 3)
<class 'bool'>
(480, 640, 3)
<class 'bool'>
(480, 640, 3)
<class 'bool'>
(480, 640, 3)
<class 'bool'>
(480, 640, 3)
<class 'bool'>
(480, 640, 3)
<class 'bool'>
(480, 640, 3)
<class 'bool'>
(480, 640, 3)
<class 'bool'>
(480, 640, 3)
<class 'bool'>
(480, 640, 3)
<class 'bool'>
(480, 640, 3)
<class 'bool'>

In [3]:
def gen_colors(num_colors):
    """Generate different colors.

    # Arguments
      num_colors: total number of colors/classes.

    # Output
      bgrs: a list of (B, G, R) tuples which correspond to each of
            the colors/classes.
    """
    import random
    import colorsys

    hsvs = [[float(x) / num_colors, 1., 0.7] for x in range(num_colors)]
    random.seed(1234)
    random.shuffle(hsvs)
    rgbs = list(map(lambda x: list(colorsys.hsv_to_rgb(*x)), hsvs))
    bgrs = [(int(rgb[2] * 255), int(rgb[1] * 255),  int(rgb[0] * 255))
            for rgb in rgbs]
    return bgrs
gen_colors(3)

[(178, 0, 0), (0, 0, 178), (0, 178, 0)]

In [35]:
from lxml import etree
class gen_Annotations:
    def __init__(self, filename):
        self.root = etree.Element('annotation')
        child1 = etree.SubElement(self.root, 'filename')
        child1.text = filename
        
        child2 = etree.SubElement(self.root, 'source')
        child3 = etree.SubElement(child2, 'database')
        child3.text = 'PASCAL: safety tools of substation'
        
    def set_size(self, width, height, channel):
        size = etree.SubElement(self.root, 'size')
        widthn = etree.SubElement(size, 'width')
        widthn.text = str(width)
        heightn = etree.SubElement(size, 'height')
        heightn.text = str(height)
        channeln = etree.SubElement(size, 'depth')
        channeln.text = str(channel)
    def add_pic_attr(self, label, xmin, ymin, xmax, ymax):
        object = etree.SubElement(self.root, 'object')
        namen = etree.SubElement(object, 'name')
        namen.text = label
        bndbox = etree.SubElement(object, 'bndbox')
        xminn = etree.SubElement(bndbox, 'xmin')
        xminn.text = str(xmin)
        yminn = etree.SubElement(bndbox, 'ymin')
        yminn.text = str(ymin)
        
        xmaxn = etree.SubElement(bndbox, 'xmax')
        xmaxn.text = str(xmax)
        ymaxn = etree.SubElement(bndbox, 'ymax')
        ymaxn.text = str(ymax)
    def save_xml(self, filename):
        tree = etree.ElementTree(self.root)
        tree.write(filename, pretty_print=True, xml_declaration=False, encoding='utf-8')
        
if __name__ == '__main__':
    filename = '00001,jpg'
    anno = gen_Annotations(filename)
    anno.set_size(1440,1080, 3)
    for i in range(3):
        xmin =i+1
        ymin = i+10
        xmax = i+50
        ymax = i+50
        anno.add_pic_attr('pencil', xmin, ymin, xmax, ymax)
    anno.save_xml('/home/cym/CYM/00001.xml')

AttributeError: module 'xml.etree' has no attribute 'Element'

In [46]:
import xml.etree.ElementTree as et
import cv2
def get_xml_annotation(xml_path, img_path):
    tree = et.ElementTree()
    tree.parse(xml_path)
    root= tree.getroot()
    source = et.SubElement(root, 'source')
    database = et.SubElement(source, 'database')
    database.text = 'PASCAL: safety tools of substation'
    
    img = cv2.imread(img_path)
    shape = img.shape # (height, width, channel)
    print(shape)
    size = et.SubElement(root, 'size')
    width = et.SubElement(size, 'width')
    width.text = str(shape[1])
    height = et.SubElement(size, 'height')
    height.text = str(shape[0])
    depth = et.SubElement(size, 'depth')
    depth.text = str(shape[2])
    
    objects = tree.findall('object')
    num_obj = len(objects)
    print('num_obj=', num_obj)
    for i in range(num_obj):
        obj = objects[i]
        label = obj.find('label')
        cls_name = label.text
        obj.remove(label)
        name = et.SubElement(obj, 'name')
        name.text = cls_name
        pose = et.SubElement(obj, 'pose')
        pose.text = 'Unspecified'
        difficult = et.SubElement(obj, 'difficult')
        difficult.text = str(0)
        truncated = et.SubElement(obj, 'truncated')
        truncated.text = str(0)
    tree.write('/home/cym/桌面/图像标注/test.xml', xml_declaration=False, encoding='utf-8')
xml_path = '/home/cym/桌面/图像标注/原图/ESR-F/ESR-F-XML（验电操作-负样本）/\
identification cards（标识牌）/ESR-F-01-0010_0001.xml'
img_path = '/home/cym/桌面/图像标注/原图/ESR-F/ESR-F/ESR-F-01-0010_0001.jpg'
get_xml_annotation(xml_path, img_path)

(1080, 1440, 3)
num_obj= 17


In [70]:
import xml.etree.ElementTree as et
import xml.etree as etree
import cv2
import glob
def get_xml_annotation(xml_path, img_path):  
    img_name = img_path.split('/')[-1]
    new_tree_root = et.Element('annotation')
    
    xml_tree = et.ElementTree()
    xml_tree.parse(xml_path)
    root= xml_tree.getroot()
    
    filename = xml_tree.find('filename')
#     filename.text = img_name 
    new_tree_root.append(filename)
    
    if img_path.split('/')[-1].split('.')[0] != xml_path.split('/')[-1].split('.')[0] or img_path.split('/')[-1] != filename.text:
        print(img_path)
        print(xml_path)
        print(filename.text)
#         return 
    
    source = et.SubElement(root, 'source')
    database = et.SubElement(source, 'database')
    database.text = 'PASCAL: safety tools of substation'
    new_tree_root.append(source)
    
    img = cv2.imread(img_path)
    shape = img.shape # (height, width, channel)
#     print(shape)
    size = et.SubElement(root, 'size')
    width = et.SubElement(size, 'width')
    width.text = str(shape[1])
    height = et.SubElement(size, 'height')
    height.text = str(shape[0])
    depth = et.SubElement(size, 'depth')
    depth.text = str(shape[2])
    new_tree_root.append(size)
    
    
    objects = xml_tree.findall('object')
    num_obj = len(objects)
#     print('num_obj=', num_obj)
    for i in range(num_obj):
        obj = objects[i]
        label = obj.find('label')
        cls_name = label.text
        obj.remove(label)
        name = et.SubElement(obj, 'name')
        name.text = cls_name
        pose = et.SubElement(obj, 'pose')
        pose.text = 'Unspecified'
        difficult = et.SubElement(obj, 'difficult')
        difficult.text = str(0)
        truncated = et.SubElement(obj, 'truncated')
        truncated.text = str(0)
        new_tree_root.append(obj)
    t = et.ElementTree(new_tree_root)
    
    save_path = '/home/cym/桌面/图像标注/原图/ESR-F/Annotaions_from_iden/'+img_name.split('.')[0]+'.xml'
    t.write(save_path, xml_declaration=False, encoding='utf-8')
xml_dir = '/home/cym/CYM/dataset/变电站安监/第一批交付成果/图像标注/标注/ESR-F-XML（验电操作-负样本）\
/identification cards（标识牌）/'
img_dir = '/home/cym/桌面/图像标注/原图/ESR-F/ESR-F/'

xml_paths = sorted(glob.glob(xml_dir+'**.xml'))
img_paths = sorted(glob.glob(img_dir+'**.jpg'))
print(len(xml_paths))
print(len(img_paths))
for i in range(len(xml_paths)):
    xml_path = xml_paths[i]
    img_path = img_paths[i]
    get_xml_annotation(xml_path, img_path)

1105
1105
/home/cym/桌面/图像标注/原图/ESR-F/ESR-F/ESR-F-01-0081_0011.jpg
/home/cym/CYM/dataset/变电站安监/第一批交付成果/图像标注/标注/ESR-F-XML（验电操作-负样本）/identification cards（标识牌）/ESR-F-01-0081_0011.xml
ESR-T-01-0081_0011.jpg
/home/cym/桌面/图像标注/原图/ESR-F/ESR-F/ESR-F-02-0031_0005.jpg
/home/cym/CYM/dataset/变电站安监/第一批交付成果/图像标注/标注/ESR-F-XML（验电操作-负样本）/identification cards（标识牌）/ESR-F-02-0031_0005.xml
ESR-T-02-0031_0005.jpg
/home/cym/桌面/图像标注/原图/ESR-F/ESR-F/ESR-F-02-0033_0007.jpg
/home/cym/CYM/dataset/变电站安监/第一批交付成果/图像标注/标注/ESR-F-XML（验电操作-负样本）/identification cards（标识牌）/ESR-F-02-0033_0007.xml
ESR-T-02-0033_0004.jpg
/home/cym/桌面/图像标注/原图/ESR-F/ESR-F/ESR-F-02-0145_0009.jpg
/home/cym/CYM/dataset/变电站安监/第一批交付成果/图像标注/标注/ESR-F-XML（验电操作-负样本）/identification cards（标识牌）/ESR-F-02-0145_0009.xml
ESR-T-02-0145_0009.jpg
/home/cym/桌面/图像标注/原图/ESR-F/ESR-F/ESR-F-03-0063_0007.jpg
/home/cym/CYM/dataset/变电站安监/第一批交付成果/图像标注/标注/ESR-F-XML（验电操作-负样本）/identification cards（标识牌）/ESR-F-03-0063_0007.xml
ESR-T-03-0063_0004.jpg
/home/cym/桌面/图像标注/原图/ESR-F/ESR

/home/cym/桌面/图像标注/原图/ESR-F/ESR-F/ESR-F-03-0224_0009.jpg
/home/cym/CYM/dataset/变电站安监/第一批交付成果/图像标注/标注/ESR-F-XML（验电操作-负样本）/identification cards（标识牌）/ESR-F-03-0224_0009.xml
ESR-T-03-0224_0009.jpg
/home/cym/桌面/图像标注/原图/ESR-F/ESR-F/ESR-F-03-0227_0002.jpg
/home/cym/CYM/dataset/变电站安监/第一批交付成果/图像标注/标注/ESR-F-XML（验电操作-负样本）/identification cards（标识牌）/ESR-F-03-0227_0002.xml
ESR-T-03-0227_0002.jpg
/home/cym/桌面/图像标注/原图/ESR-F/ESR-F/ESR-F-03-0227_0003.jpg
/home/cym/CYM/dataset/变电站安监/第一批交付成果/图像标注/标注/ESR-F-XML（验电操作-负样本）/identification cards（标识牌）/ESR-F-03-0227_0003.xml
ESR-T-03-0227_0003.jpg
/home/cym/桌面/图像标注/原图/ESR-F/ESR-F/ESR-F-03-0227_0004.jpg
/home/cym/CYM/dataset/变电站安监/第一批交付成果/图像标注/标注/ESR-F-XML（验电操作-负样本）/identification cards（标识牌）/ESR-F-03-0227_0004.xml
ESR-T-03-0227_0004.jpg
/home/cym/桌面/图像标注/原图/ESR-F/ESR-F/ESR-F-03-0227_0007.jpg
/home/cym/CYM/dataset/变电站安监/第一批交付成果/图像标注/标注/ESR-F-XML（验电操作-负样本）/identification cards（标识牌）/ESR-F-03-0227_0007.xml
ESR-T-03-0227_0007.jpg
/home/cym/桌面/图像标注/原图/ESR-F/ESR-F/ESR-F-0

/home/cym/桌面/图像标注/原图/ESR-F/ESR-F/ESR-F-04-0077_0004.jpg
/home/cym/CYM/dataset/变电站安监/第一批交付成果/图像标注/标注/ESR-F-XML（验电操作-负样本）/identification cards（标识牌）/ESR-F-04-0077_0004.xml
ESR-T-04-0077_0004.jpg
/home/cym/桌面/图像标注/原图/ESR-F/ESR-F/ESR-F-04-0077_0008.jpg
/home/cym/CYM/dataset/变电站安监/第一批交付成果/图像标注/标注/ESR-F-XML（验电操作-负样本）/identification cards（标识牌）/ESR-F-04-0077_0008.xml
ESR-T-04-0077_0003.jpg
/home/cym/桌面/图像标注/原图/ESR-F/ESR-F/ESR-F-04-0080_0003.jpg
/home/cym/CYM/dataset/变电站安监/第一批交付成果/图像标注/标注/ESR-F-XML（验电操作-负样本）/identification cards（标识牌）/ESR-F-04-0080_0003.xml
ESR-T-04-0080_0003.jpg
/home/cym/桌面/图像标注/原图/ESR-F/ESR-F/ESR-F-04-0080_0004.jpg
/home/cym/CYM/dataset/变电站安监/第一批交付成果/图像标注/标注/ESR-F-XML（验电操作-负样本）/identification cards（标识牌）/ESR-F-04-0080_0004.xml
ESR-T-04-0080_0004.jpg
/home/cym/桌面/图像标注/原图/ESR-F/ESR-F/ESR-F-04-0080_0005.jpg
/home/cym/CYM/dataset/变电站安监/第一批交付成果/图像标注/标注/ESR-F-XML（验电操作-负样本）/identification cards（标识牌）/ESR-F-04-0080_0005.xml
ESR-T-04-0080_0005.jpg
/home/cym/桌面/图像标注/原图/ESR-F/ESR-F/ESR-F-0